In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import random
import torch
import evaluate
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Load dataset, model and tokenizer of model
dataset_name = "ccdv/pubmed-summarization"
model_name = "google/flan-t5-small"

# Load the dataset and select the first 10,000 records from the 'train' split
dataset = load_dataset(dataset_name)
first_10k_train = dataset['train'].select(range(400))

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
random.seed(42)

# Define a function to preprocess the dataset
def preprocess_function(batch):
    # Assuming 'article' and 'abstract' are lists of strings for each example in the batch
    inputs = [f"summarize: {article}" for article in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(batch["abstract"], max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Check if predictions is a tuple
    if isinstance(predictions, tuple):
        # If predictions is a tuple, we assume the first element is the logits
        predictions = predictions[0]
    # Convert logits to token IDs
    predictions = np.argmax(predictions, axis=-1)
    # Ensure predictions is a 2D list
    predictions = predictions.tolist()
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Multiply each value in the result dictionary by 100
    result = {key: round(value * 100, 4) for key, value in result.items()}

    return result

# Tokenize and preprocess the dataset
tokenized_dataset_train = first_10k_train.map(preprocess_function, batched=True)
tokenized_dataset_validate = dataset["validation"].select(range(400)).map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=1000,
    evaluation_strategy="epoch",
    eval_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    greater_is_better=True,
    save_strategy="epoch",
    save_total_limit=2,
    learning_rate=2e-5,
)

# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    data_collator=data_collator,
    eval_dataset=tokenized_dataset_validate,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# Train the model
print("Starting to train...")
trainer.train()

# Save the model and tokenizer
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_tokenizer")


/home/student/anaconda3/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for ccdv/pubmed-summarization contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/pubmed-summarization
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/home/student/anaconda3/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this 

Starting to train...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: ojaswini-kohale (opnp). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.993900,3.499626,48.170900,20.186400,41.023800,41.069600
2,3.067900,3.398567,48.388600,20.791100,41.526700,41.581600
3,3.499900,3.303023,48.526000,21.231600,41.876800,41.936700
4,3.312300,3.195276,48.543800,21.499600,41.966200,42.000800
5,2.815600,3.089887,48.620700,21.684600,42.042800,42.093700


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


('./saved_tokenizer/tokenizer_config.json',
 './saved_tokenizer/special_tokens_map.json',
 './saved_tokenizer/spiece.model',
 './saved_tokenizer/added_tokens.json',
 './saved_tokenizer/tokenizer.json')

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import random
import torch
import evaluate
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

batch_size = 400

total_batches = len(dataset['train'])

tokenizer = AutoTokenizer.from_pretrained("saved_tokenizer")
model = AutoModelForSeq2SeqLM.from_pretrained("saved_model")
random.seed(42)

# Define a function to preprocess the dataset
def preprocess_function(batch):
    # Assuming 'article' and 'abstract' are lists of strings for each example in the batch
    inputs = [f"summarize: {article}" for article in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(batch["abstract"], max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Check if predictions is a tuple
    if isinstance(predictions, tuple):
        # If predictions is a tuple, we assume the first element is the logits
        predictions = predictions[0]
    # Convert logits to token IDs
    predictions = np.argmax(predictions, axis=-1)
    # Ensure predictions is a 2D list
    predictions = predictions.tolist()
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Multiply each value in the result dictionary by 100
    result = {key: round(value * 100, 4) for key, value in result.items()}

    return result

for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    current_batch = dataset['train'].select(range(start_idx, end_idx))
    tokenized_dataset_train = current_batch.map(preprocess_function, batched=True)
    tokenized_dataset_validate = dataset["validation"].select(range(400)).map(preprocess_function, batched=True)
    
    # Define training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        num_train_epochs=5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        save_steps=1000,
        evaluation_strategy="epoch",
        eval_steps=1000,
        load_best_model_at_end=True,
        metric_for_best_model="rouge1",
        greater_is_better=True,
        save_strategy="epoch",
        save_total_limit=2,
        learning_rate=2e-5,
    )
    
    # Create a data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    # Initialize the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset_train,
        data_collator=data_collator,
        eval_dataset=tokenized_dataset_validate,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    # Train the model
    print("Starting to train...")
    trainer.train()

    # Save the model and tokenizer
    trainer.save_model("./saved_model")
    tokenizer.save_pretrained("./saved_tokenizer")


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/home/student/anaconda3/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Starting to train...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.685200,3.082449,48.656700,21.689200,42.110600,42.116800
2,2.802800,3.045423,48.549100,21.612300,41.972200,41.980800
3,3.077500,2.975374,48.463200,21.619400,41.965800,41.992700
4,3.021700,2.867026,48.351000,21.675200,41.913400,41.940500
5,2.668100,2.749565,48.358100,21.700000,41.910500,41.945600


...

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/home/student/anaconda3/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Starting to train...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.555600,2.160083,50.785000,23.643700,44.525700,44.567100
2,2.575900,2.159645,50.794800,23.638400,44.548500,44.585000
3,2.371500,2.159388,50.727200,23.594700,44.464000,44.522500
4,2.596900,2.161475,50.684600,23.596000,44.403200,44.450600


Checkpoint destination directory ./results/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [ ]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import random
import evaluate
import nltk
import torch
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")


# Load dataset, model and tokenizer of model
dataset_name = "ccdv/pubmed-summarization"
model_name = "google/flan-t5-large"
dataset = load_dataset(dataset_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch.float16)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float16)
random.seed(42)
# Unused columns for fine-tuning
remove_columns=["article", "abstract"]

first_10k_train = dataset['train'].select(range(10000))

# Get the maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
def get_max_length(dataset, column_key, tokenizer):
    tokenized_data = concatenate_datasets([dataset["train"], dataset["test"]]).map(
        lambda x: tokenizer(x[column_key], truncation=True), 
        batched=True, 
        remove_columns=remove_columns
    )
    max_length = max([len(x) for x in tokenized_data["input_ids"]])
    return max_length

max_source_length = get_max_length(dataset, "article", tokenizer)
print(f"Max source sequence length: {max_source_length}")

max_target_length = get_max_length(dataset, "abstract", tokenizer)
print(f"Max target sequence length: {max_target_length}")

prefix = "summarize: "

def preprocess_function(examples, padding="max_length"):
    
    # Add prefix to the input for Flan-T5
    inputs = [prefix + example for example in examples["article"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=examples["abstract"], max_length=max_target_length, padding=padding, truncation=True)

    # Replace all tokenizer.pad_token_id in the labels by -100 because we want to ignore padding in the loss
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=remove_columns)
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

# Load ROUGE metric
metric = evaluate.load("rouge")

# Function for post-processing text
def postprocess_text(predictions, labels):
    # Remove spaces at the beginning and at the end of texts
    predictions = [prediction.strip() for prediction in predictions]
    labels = [label.strip() for label in labels]

    # Operate texts at the level of sentences
    predictions = ["\n".join(sent_tokenize(prediction)) for prediction in predictions]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return predictions, labels


def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    if isinstance(predictions, tuple):
        predictions = predictions[0]
        
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)

    result = metric.compute(predictions=decoded_predictions, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(prediction != tokenizer.pad_token_id) for prediction in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return result


# Define data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    #pad_to_multiple_of=8
)

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="sft-flan-t5",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=2,
    logging_dir="sft-flan-t5/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Create Seq2SeqTrainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

# Training model
print("Starting to train...")
trainer.train()
trainer.save_model("saved_model")
tokenizer.save_pretrained("saved_tokenizer")



In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("saved_model")
tokenizer = AutoTokenizer.from_pretrained("saved_tokenizer")
validation_dataset = load_dataset(dataset_name)["validation"]
validation_dataset = validation_dataset.map(preprocess_function, batched=True)

In [ ]:
!pip install tqdm
from tqdm import tqdm

model.eval()
generated_summaries = []
for batch in tqdm(validation_dataset, desc="Generating summaries"):
    with torch.no_grad():
        inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True)
        outputs = model.generate(**inputs)
        summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        generated_summaries.extend(summaries)

In [ ]:
from evaluate import load

# Load the ROUGE metric
rouge = load("rouge")

# Assuming generated_summaries and references are already defined
results = rouge.compute(predictions=generated_summaries, references=references, use_stemmer=True)

# Adjusted to directly use the numpy.float64 values
results = {key: value * 100 for key, value in results.items()}
print(results)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = "saved_model"
tokenizer_path = "saved_tokenizer"
max_source_length =512
max_target_length = 512
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

text = "A 47-year-old female patient presents with a history of abdominal pain that began three days ago, localized to the lower right quadrant of the abdomen, and characterized by sharp, stabbing sensations that worsen with movement and after meals. The pain is relieved by rest and warmth but aggravates with movement and eating. The patient reports that the pain radiates to the lower back and is worse in the morning and after meals, rating the severity as a 7 out of 10. Vital signs are within normal limits, and the physical examination reveals tenderness in the lower right quadrant of the abdomen. Laboratory data, including a complete blood count (CBC) and electrolyte panel, are pending, and an abdominal ultrasound is scheduled for tomorrow. The patient's problem list includes abdominal pain, tenderness in the lower right quadrant, pain worsening with movement and after meals, and radiating pain to the lower back. The plan involves diagnostic testing, including a complete blood count (CBC) and electrolyte panel to rule out anemia and electrolyte imbalances, and an abdominal ultrasound to identify the cause of the pain. Therapy includes pain management with over-the-counter analgesics and instructions to avoid strenuous activity and to rest. A consultation with a gastroenterologist is recommended for further evaluation. The patient is educated on the importance of adhering to the prescribed medication regimen and the benefits of dietary changes, such as avoiding spicy foods and drinking plenty of water."
inputs = tokenizer(text, return_tensors="pt", max_length=max_source_length, truncation=True)

summary_ids = model.generate(inputs["input_ids"], max_length=max_target_length, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

In [ ]:














`